# 3. Prompt Chaining with OpenAI Responses API

Prompt Chaining is a technique in LLM workflows where you break a complex task into multiple smaller steps, each handled by its own prompt.
The output of one prompt becomes the input to the next, forming a chain.



#### 🧠 Why use Prompt Chaining?

LLMs struggle with long, multi-step reasoning in a single prompt.
By chaining prompts, you:

| Benefit              | Meaning                              |
| -------------------- | ------------------------------------ |
| Decompose complexity | Handle one step at a time            |
| Improve accuracy     | Fewer hallucinations                 |
| Add structure        | Each step has a clear responsibility |
| Enable modularity    | Easy to replace or improve one step  |

Prompt 1 → Output A → Prompt 2 → Output B → Prompt 3 → Final Result

This notebook shows how to build **step‑by‑step prompt chains** using the modern `client.responses.create()` API.
We will:
- Configure the OpenAI client and model
- Send a single prompt
- Build a **two‑step chain** (outline → expand)
- Build a **multi‑step chain** with a small helper function

In [2]:
# ✅ Step 0 — Imports & client setup
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


In [4]:




# Load API key from environment variable OPENAI_API_KEY
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")  # make sure this is set in your environment
)

# Global model config — change here to switch models everywhere
model_name = "gpt-4o-mini"  # e.g. "gpt-4o", "gpt-4.1-mini", etc.

In [5]:
# ✅ Helper: small wrapper so we don't repeat boilerplate
def run_llm(user_message: str, system_message: str | None = None):
    """Send a simple 1‑turn request and return output_text."""
    messages = []
    if system_message:
        messages.append({"role": "system", "content": system_message})
    messages.append({"role": "user", "content": user_message})

    response = client.responses.create(
        model=model_name,
        input=messages,
        temperature=0.3,
    )
    return response.output_text


## Step 1 — Single prompt

Start with a single question to the model using `run_llm`.

In [6]:
single_answer = run_llm(
    user_message="Explain prompt chaining in 3 bullet points.",
    system_message="You are a concise AI tutor.",
)
print(single_answer)

- **Definition**: Prompt chaining involves using the output of one prompt as the input for the next, creating a sequence that builds on previous responses to generate more complex or refined results.

- **Purpose**: This technique enhances the depth and coherence of generated content, allowing for more nuanced and contextually aware interactions, especially in tasks requiring multi-step reasoning.

- **Application**: Commonly used in creative writing, problem-solving, and programming tasks, prompt chaining helps in developing narratives, solving complex queries, or iterating on ideas effectively.


## Step 2 — Two‑step prompt chain (outline → expand)

We first ask the model to generate an **outline**, then feed that outline back in a second call
to generate a detailed explanation.


In [7]:
# Step 2a — Ask for an outline
outline = run_llm(
    user_message=(
        "Create a short outline for a 5‑minute talk on 'Prompt Chaining for Beginners'. "
        "Use 4–5 bullet points."
    ),
    system_message="You are a helpful tech speaker coach.",
)
print("--- OUTLINE ---\n", outline)

--- OUTLINE ---
 ### Outline for a 5-Minute Talk on 'Prompt Chaining for Beginners'

1. **Introduction to Prompt Chaining**
   - Define prompt chaining and its significance in AI interactions.
   - Briefly explain how it enhances the effectiveness of AI responses.

2. **Basic Concepts of Prompt Chaining**
   - Explain the concept of prompts and how they guide AI.
   - Introduce the idea of linking prompts to build context.

3. **Step-by-Step Process**
   - Demonstrate how to create a simple prompt chain.
   - Provide an example of a basic prompt followed by a follow-up prompt.

4. **Best Practices for Effective Prompt Chaining**
   - Tips for clarity and specificity in prompts.
   - Importance of iterative refinement to improve results.

5. **Conclusion and Q&A**
   - Recap the benefits of prompt chaining.
   - Invite questions and encourage experimentation with prompt chaining in AI applications.


In [8]:
# Step 2b — Use the outline as input to expand into a mini talk script
expanded_script = run_llm(
    user_message=(
        "You are given this outline for a 5‑minute talk on prompt chaining:\n\n"
        f"{outline}\n\n"
        "Turn it into a short script in simple English. Keep it under 500 words."
    ),
    system_message="You write clear, friendly technical explanations.",
)
print("--- EXPANDED SCRIPT ---\n", expanded_script)

--- EXPANDED SCRIPT ---
 ### Script for a 5-Minute Talk on 'Prompt Chaining for Beginners'

**Introduction to Prompt Chaining**

Hello everyone! Today, we’re going to talk about something called prompt chaining. So, what is prompt chaining? Simply put, it’s a way to connect different prompts when you’re interacting with AI. This technique helps the AI understand your questions better and gives you more useful answers. By linking prompts, we can create a clearer context for the AI, making our conversations smoother and more effective.

**Basic Concepts of Prompt Chaining**

Let’s break this down a bit. First, what is a prompt? A prompt is basically a question or instruction you give to the AI to get a response. Think of it as a way to guide the AI in the direction you want it to go. Now, when we talk about prompt chaining, we mean linking these prompts together. This helps build context, so the AI can follow your train of thought and give more relevant answers.

**Step-by-Step Process**

## Step 3 — Generalising prompt chaining

Let's build a small **prompt chain helper** that:
- Takes an initial user query
- Runs a series of *steps* (functions) that each call the model
- Passes intermediate results forward

We will implement a 3‑step chain:
1. Rewrite the original question more clearly
2. Break it into sub‑questions
3. Answer all sub‑questions and summarise


In [11]:
def step_rewrite_question(original_question: str) -> str:
    return run_llm(
        user_message=(
            "Rewrite this question to be clearer and more precise. "
            "Keep the *same intent*:\n\n" + original_question
        ),
        system_message="You are an expert at clarifying user questions.",
    )


def step_decompose_question(clear_question: str) -> str:
    return run_llm(
        user_message=(
            "Break this question into 3–5 smaller sub‑questions that, when answered, "
            "fully address the original question. Number the sub‑questions.\n\n"
            + clear_question
        ),
        system_message="You are good at problem decomposition.",
    )


def step_answer_subquestions(clear_question: str, sub_questions: str) -> str:
    return run_llm(
        user_message=(
            "You are given an original question and a set of numbered sub‑questions.\n\n"
            f"Original question: {clear_question}\n\n"
            f"Sub‑questions: \n{sub_questions}\n\n"
            "Answer each sub‑question briefly, then provide a 1‑paragraph overall answer."
        ),
        system_message="You are an expert tutor who answers step‑by‑step.",
    )

In [12]:
# Putting it all together: a generic chaining function
def run_prompt_chain(original_question: str) -> dict:
    """Run a 3‑step chain and return all intermediate steps."""
    clear_q = step_rewrite_question(original_question)
    subs = step_decompose_question(clear_q)
    final_answer = step_answer_subquestions(clear_q, subs)

    return {
        "original_question": original_question,
        "clear_question": clear_q,
        "sub_questions": subs,
        "final_answer": final_answer,
    }


In [13]:
# Try the chain
question = "How can I use prompt chaining to improve my RAG system's answers?"
chain_result = run_prompt_chain(question)

for key, value in chain_result.items():
    print("=====", key, "=====")
    print(value)
    print()

===== original_question =====
How can I use prompt chaining to improve my RAG system's answers?

===== clear_question =====
How can I implement prompt chaining to enhance the responses of my RAG system?

===== sub_questions =====
To address the question of implementing prompt chaining to enhance the responses of your RAG (Retrieval-Augmented Generation) system, consider the following sub-questions:

1. **What is prompt chaining, and how does it work in the context of RAG systems?**
2. **What are the specific goals or improvements I want to achieve by using prompt chaining in my RAG system?**
3. **What types of prompts should I design for each stage of the chaining process to maximize the quality of responses?**
4. **How can I evaluate the effectiveness of the prompt chaining implementation on the responses generated by my RAG system?**
5. **What tools or frameworks can assist in the implementation and testing of prompt chaining within my RAG system?**

===== final_answer =====
### 1. W